<div align="center">
    <img src="logo/logo_warren_terracota.webp" alt="Logo Warren Terracota" width="400"> </div>
</div>

**Solução por Wallace Santo.**

In [ ]:
#Importando bibliotecas iniciais
import yfinance as yf # Para obter os dados do Yahoo Finance
import pandas as pd # Para utilizar os dataframes
import matplotlib.pyplot as plt #Criar gráficos


In [ ]:
# Datas no formato americano(compatível com o Yahoo Finance)
data_inicio = "2024-02-20"
data_final = "2024-04-09"

# Definindo o código do ativo na bolsa
codigo = "LFTS11.SA"

# Baixando os dados históricos do ativo
historico_lfts11 = yf.download(codigo, start=data_inicio, end=data_final)

# Removendo 'Adj Close', por ser indexado em Selic, não há diferença real entre Close e Adj Close. 
# Removendo 'Volume', pois o algoritmo TWAP não utiliza essa métrica
historico_lfts11 = historico_lfts11.drop(columns=['Adj Close', 'Volume'])

# Criando uma coluna com a média de 'Open', 'High', 'low' e 'Close' de cada dia
historico_lfts11['Mean'] = historico_lfts11.mean(axis=1)

# Visualiazndo o histórico
historico_lfts11


In [ ]:
# Calculando o TWAP
twap = historico_lfts11['Mean'].mean()
twap

In [ ]:
# Abrindo a planilha do desafio para automação
automacao = pd.read_excel('planilha_de_saldo_para_automao.xlsx')


In [ ]:
# Importando a biblioteca math para trabalhamos apenas com valores inteiros
import math

# Definindo o valor de compra ou venda para o TWAP encontrado anteriormente
valor_lfts11 = twap

# Função para calcular a operação de compra ou venda
def operacao(saldo):
    #Verificando se o saldo é negativo
    if saldo < 0:
        quantidade_venda = abs(saldo) / valor_lfts11
        #Usando a função teto para garantir todos os casos
        quantidade_venda = math.ceil(quantidade_venda)
        return f"Vender {quantidade_venda} cota(s)"
    else:
        quantidade_compra = saldo / valor_lfts11
        #Usando a função piso para garantir que não fique com saldo negativo
        quantidade_compra = math.floor(quantidade_compra)
        return f"Comprar {quantidade_compra} cota(s)"

# Aplicando a função no dataframe
automacao['Operação'] = automacao['Vl. Total'].apply(operacao)

automacao


In [ ]:
# Separando a nossa operação em Tipo(Compra ou Venda), Quantidade e Unidade
automacao['Tipo'], automacao['Quantidade'], automacao['Unidade'] = zip(*automacao['Operação'].str.split().tolist())

# Retirando as colunas Operação e Unidade para deixar o DataFrame mais limpo
automacao = automacao.drop(columns=['Operação', 'Unidade'])

automacao




In [ ]:
# Abrindo o modelo_mesa que é o exemplo para o desafio
modelo_mesa = pd.read_excel('modelo_mesa_-_lfts11_-_exemplo_-_desafio_vba.xlsx')

In [ ]:
# Visualizando o DataFrame
modelo_mesa

In [ ]:
# Adicionando cada linha do DataFrame 'automacao', que era o desafio
for index, row in automacao.iterrows():
    # Definir se é C ou V com base na coluna 'Tipo'
    cv = 'C' if 'Comprar' in row['Tipo'] else 'V'

    # Iterando a partir do indíce final da planilha modelo_mesa 
    numero = 481 + index

    # Criando uma linha como um DataFrame para concatenar
    nova_linha = pd.DataFrame({
        'C\V': [cv],
        'Código': ['LFTS11'],
        'Qtd': [row['Quantidade']],
        'Qtd Aberta': [None],
        'Preço': ['MERCADO'],
        'Cliente': [row['Cod. Conta Local']],  
        'Nome Cliente': [None],
        'Agente de Custódia': [None],
        'Conta de Custódia': [None],
        'Código da Carteira': [None],
        'N°': [numero]
    })
    
    
    # Concatenar a nova linha ao DataFrame da planilha modelo_mesa
    modelo_mesa = pd.concat([modelo_mesa, nova_linha], ignore_index=True)

In [ ]:
#Visualizando o DataFrame
modelo_mesa

In [ ]:
# Verificando o Dtype 
modelo_mesa.info()

In [ ]:
# Modificando os tipos para facilitar o manuseio ao exportar para o Excel
modelo_mesa['Qtd'] = pd.to_numeric(modelo_mesa['Qtd'], errors='coerce').astype(int)
modelo_mesa['Preço'] = pd.to_numeric(modelo_mesa['Preço'], errors='coerce').astype(float)



In [ ]:
# Verificando quanto foi Comprado ou Vendido, dado o TWAP

# Valor por unidade
valor_por_unidade = twap

# Filtrar apenas as linhas em que 'C\V' é igual a 'C'
compras = modelo_mesa[modelo_mesa['C\V'] == 'C']

# Somar a quantidade para todas as compras
quantidade_compras = compras['Qtd'].sum()

# Filtrar apenas as linhas em que 'C\V' é igual a 'V'
vendas = modelo_mesa[modelo_mesa['C\V'] == 'V']

# Somar a quantidade para todas as compras
quantidade_vendas = vendas['Qtd'].sum()

# Calcular o valor total de compra e venda
total_compra = quantidade_compras * valor_por_unidade
total_venda = quantidade_vendas * valor_por_unidade

# Plotar o gráfico de barras
valores = [total_compra, total_venda]
categorias = ['Compra', 'Venda']

plt.figure(figsize=(8, 6))
cores = ['#87CEEB', '#D3D3D3']  # Azul claro e cinza claro
bars = plt.bar(categorias, valores, color=cores)
plt.xlabel('Tipo de Operação')
plt.ylabel('Valor em Reais')
plt.title('Valor Total de Compra e Venda')

# Adicionar "R$" a cada número do eixo y
for bar, valor in zip(bars, valores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 'R$ {:.2f}'.format(valor), 
             va='bottom', ha='center', color='black')

plt.show()


In [ ]:
# Exportando para Excel
nome_do_arquivo = 'modelo_mesa_desafio_resolvido.xlsx'
df.to_excel(nome_do_arquivo, index=False)

#Foi necessário adicionar a formatação manualmente

### Com isso, completamos a primeira etapa do desafio, que era referente ao arquivo modelo_mesa.

### Contudo, precisamos atualizar o modelo_twap.


In [ ]:
modelo_twap = pd.read_excel('modelo_twap_-_lfts11_-_exemplo_-_desafio_vba.xlsx')

In [ ]:
modelo_twap

In [ ]:
# A nova quantidade de COMPRA é dada por quantidade_compras
# A nova quantidade de VENDA é dada por quantidade_vendas

# Alterando diretamente, teremos

modelo_twap['QUANTIDADE'] = [quantidade_compras, quantidade_vendas]

# Visualizando modelo_twap atualizado

modelo_twap

In [ ]:
# Exportando para o Excel

nome_do_arquivo = 'modelo_twap_desafio_resolvido.xlsx'
modelo_twap.to_excel(nome_do_arquivo, index=False)